In [ ]:
import os
import time
import pandas as pd
import openai
import re
import requests
import sys
from num2words import num2words
import numpy as np
from openai.embeddings_utils import get_embedding, cosine_similarity
import tiktoken

openai.api_key =  os.getenv("OPENAI_API_KEY")
openai.organization = "https://rg-openai-sandbox.openai.azure.com/"
openai.api_type = "azure"
openai.api_base = "https://rg-openai-sandbox.openai.azure.com/"
openai.api_version = "2022-12-01"

path ='./'
d = []

start_time = time.time()

for root, directories, files in os.walk(path, topdown=False):
    for file in files:
        if file.lower().endswith(".txt"):
            name = os.path.join(root, file)
            with open(name, "r", encoding="utf-8") as x:
                for line in x:
                    d.append({'FILE NAME': file, 'CONTENT': line.strip()})
                    
df = pd.DataFrame(d)
end_time = time.time()
duration = end_time - start_time

print("Script Execution:", duration)
print(df)

Script Execution: 0.013620138168334961
      FILE NAME                                            CONTENT
0    output.txt                                          CHAPTER 1
1    output.txt                             • Tuesday, September 2
2    output.txt  “Bill Palmer here,” I say, answering my cell p...
3    output.txt  I’m late, so I’m driving ten miles per hour ov...
4    output.txt  my usual  ve. I spent the morning at the docto...
..          ...                                                ...
302  output.txt  us in trouble with the union, maybe even trigg...
303  output.txt              creating some very bad press for us.”
304  output.txt  I nod automatically. “Restore critical busines...
305  output.txt            of the front-page news. Got it. anks.”
306  output.txt       Why, exactly, I’m thanking him is not clear.

[307 rows x 2 columns]


In [61]:
df = pd.DataFrame(d)
df

,FILE NAME,CONTENT
0,output.txt,CHAPTER 1
1,output.txt,"• Tuesday, September 2"
2,output.txt,"“Bill Palmer here,” I say, answering my cell p..."
3,output.txt,"I’m late, so I’m driving ten miles per hour ov..."
4,output.txt,my usual  ve. I spent the morning at the docto...
...,...,...
302,output.txt,"us in trouble with the union, maybe even trigg..."
303,output.txt,creating some very bad press for us.”
304,output.txt,I nod automatically. “Restore critical busines...
305,output.txt,of the front-page news. Got it. anks.”


In [62]:
# s is input text
def normalize_text(s, sep_token = " \n "):
    s = re.sub(r'\s+',  ' ', s).strip()
    s = re.sub(r". ,","",s)
    s = s.replace("..",".")
    s = s.replace(". .",".")
    s = s.replace("\n", "")
    s = s.replace("#","")
    s = s.strip()
    if s =="":
        s = "<blank>"
    return s
df_normalized=df.copy()
df_normalized['CONTENT'] = df["CONTENT"].apply(lambda x : normalize_text(x))

In [63]:
df_normalized


,FILE NAME,CONTENT
0,output.txt,CHAPTER 1
1,output.txt,"• Tuesday, September 2"
2,output.txt,"“Bill Palmer here,” I say, answering my cell p..."
3,output.txt,"I’m late, so I’m driving ten miles per hour ov..."
4,output.txt,my usual  ve. I spent the morning at the docto...
...,...,...
302,output.txt,"us in trouble with the union, maybe even trigg..."
303,output.txt,creating some very bad press for us.”
304,output.txt,I nod automatically. “Restore critical busines...
305,output.txt,of the front-page news. Got it. anks.”


In [64]:
df_blank = df_normalized[df_normalized.CONTENT=='<blank>']
print(df_blank)

Empty DataFrame
Columns: [FILE NAME, CONTENT]
Index: []


In [65]:
df_normalized = df_normalized[df_normalized.CONTENT !='<blank>']
len(df_normalized)
df_normalized

,FILE NAME,CONTENT
0,output.txt,CHAPTER 1
1,output.txt,"• Tuesday, September 2"
2,output.txt,"“Bill Palmer here,” I say, answering my cell p..."
3,output.txt,"I’m late, so I’m driving ten miles per hour ov..."
4,output.txt,my usual  ve. I spent the morning at the docto...
...,...,...
302,output.txt,"us in trouble with the union, maybe even trigg..."
303,output.txt,creating some very bad press for us.”
304,output.txt,I nod automatically. “Restore critical busines...
305,output.txt,of the front-page news. Got it. anks.”


In [66]:
df_normalized

,FILE NAME,CONTENT
0,output.txt,CHAPTER 1
1,output.txt,"• Tuesday, September 2"
2,output.txt,"“Bill Palmer here,” I say, answering my cell p..."
3,output.txt,"I’m late, so I’m driving ten miles per hour ov..."
4,output.txt,my usual  ve. I spent the morning at the docto...
...,...,...
302,output.txt,"us in trouble with the union, maybe even trigg..."
303,output.txt,creating some very bad press for us.”
304,output.txt,I nod automatically. “Restore critical busines...
305,output.txt,of the front-page news. Got it. anks.”


| GENERATION |TOKENIZER    | MAX INPUT TOKENS| KNOWLEDGE CUTOFF|
|------------|-------------|-----------------|-----------------|
| V2         | cl100k_base | 8191            | Sep 2021        |
| V1         | GPT-2/GPT-3 | 2046            | Aug 2020        |


https://beta.openai.com/docs/guides/embeddings/what-are-embeddings

https://openai.com/blog/new-and-improved-embedding-model/

In [67]:
tokenizer = tiktoken.get_encoding("cl100k_base")
df_tok=df_normalized.copy()
df_tok['n_tokens'] = df_normalized["CONTENT"].apply(lambda x: len(tokenizer.encode(x)))
df_tok

,FILE NAME,CONTENT,n_tokens
0,output.txt,CHAPTER 1,3
1,output.txt,"• Tuesday, September 2",6
2,output.txt,"“Bill Palmer here,” I say, answering my cell p...",19
3,output.txt,"I’m late, so I’m driving ten miles per hour ov...",19
4,output.txt,my usual  ve. I spent the morning at the docto...,24
...,...,...,...
302,output.txt,"us in trouble with the union, maybe even trigg...",15
303,output.txt,creating some very bad press for us.”,8
304,output.txt,I nod automatically. “Restore critical busines...,13
305,output.txt,of the front-page news. Got it. anks.”,15


In [9]:

# Based on https://openai.com/api/pricing/ on 01/29/2023
# If you were using this for approximating pricing with Azure OpenAI adjust the values below with: https://azure.microsoft.com/pricing/details/cognitive-services/openai-service/

#MODEL	USAGE
#Ada     v1	$0.0040 / 1K tokens
#Babbage v1	$0.0050 / 1K tokens
#Curie   v1	$0.0200 / 1K tokens
#Davinci v1	$0.2000 / 1K tokens

#MODEL	USAGE
#Ada     v2	$0.0004 / 1K tokens
#This Ada model, text-embedding-ada-002, is a better and lower cost replacement for our older embedding models. 

n_tokens_sum = df_tok['n_tokens'].sum()

ada_v1_embeddings_cost = (n_tokens_sum/1000) *.0040
babbage_v1_embeddings_cost = (n_tokens_sum/1000) *.0050
curie_v1_embeddings_cost = (n_tokens_sum/1000) *.02
davinci_v1_embeddings_cost = (n_tokens_sum/1000) *.2

ada_v2_embeddings_cost = (n_tokens_sum/1000) *.0004

print("Number of tokens: " + str(n_tokens_sum) + "\n")

print("MODEL        VERSION    COST")
print("-----------------------------------")
print("Ada" + "\t\t" + "v1" + "\t$" + '%.8s' % str(ada_v1_embeddings_cost))
print("Babbage" + "\t\t" + "v1" + "\t$" + '%.8s' % str(babbage_v1_embeddings_cost))
print("Curie" + "\t\t" + "v1" + "\t$" + '%.8s' % str(curie_v1_embeddings_cost))
print("Davinci" + "\t\t" + "v1" + "\t$" + '%.8s' % str(davinci_v1_embeddings_cost))
print("Ada" + "\t\t" + "v2" + "\t$" + '%.8s' %str(ada_v2_embeddings_cost))

Number of tokens: 2532696

MODEL        VERSION    COST
-----------------------------------
Ada		v1	$10.13078
Babbage		v1	$12.66348
Curie		v1	$50.65392
Davinci		v1	$506.5392
Ada		v2	$1.013078


In [41]:
len(df)

1

In [56]:
df_tok = df_tok[df_tok.n_tokens<400]

len(df_tok)

307

# OpenAI Text & Embedding Rate Limits?

Rate limits are enforced at the **organization level, not user level**, based on the specific endpoint used as well as the type of account you have. 

Rate limits are measured in two ways: **RPM (requests per minute)** and **TPM (tokens per minute)**. 

## TEXT & EMBEDDING

Free trial users •20 RPM •150,000 TPM

Pay-as-you-go users (first 48 hours)	•60 RPM •250,000 TPM

Pay-as-you-go users (after 48 hours)	•3,000 RPM •250,000 TPM

https://beta.openai.com/docs/guides/rate-limits/overview

In [69]:
import time
from IPython.display import clear_output

request_counter = 0
total_requests_sent = 0
rate_limit= 3000

start_timer=time.time()

def generate_embeddings(text, model="embedding"):
    global request_counter
    global rate_limit
    global total_requests_sent
    global start_timer
    clear_output(wait=True)
    check_timer = time.time()
    duration = check_timer-start_timer
    print(duration)
    
    if int(duration) >= 60:
        start_timer=time.time()
        request_counter=0
    if request_counter == rate_limit and int(duration) <= 59:
        sleep_for = 60-int(duration)
        print("Sleeping for " + str(sleep_for) +" seconds")
        print("Total requests sent: ", total_requests_sent)
        time.sleep(sleep_for)
        start_timer = time.time()
        request_counter =0
    if request_counter < rate_limit:
        request_counter+=1
        total_requests_sent+=1
        print("Request counter: ", request_counter)
        print("Total requests sent: ", total_requests_sent)
        
    return openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']

df_embeddings = df_tok.copy()
#df_bills['ada_v2'] = df_bills["text"].apply(lambda x : get_embedding(x, engine = 'text-embedding-ada-002')) # engine should be set to the deployment name you chose when you deployed the text-embedding-ada-002 (Version 2) model
df_embeddings['ada_v2_embedding'] = df_tok['CONTENT'].apply(lambda x: get_embedding(x, engine='embedding'))
df_embeddings


,FILE NAME,CONTENT,n_tokens,ada_v2_embedding
0,output.txt,CHAPTER 1,3,"[0.015155577100813389, -0.01603473164141178, -..."
1,output.txt,"• Tuesday, September 2",6,"[-0.005716212093830109, -0.010315271094441414,..."
2,output.txt,"“Bill Palmer here,” I say, answering my cell p...",19,"[-0.03608514368534088, -0.009047702886164188, ..."
3,output.txt,"I’m late, so I’m driving ten miles per hour ov...",19,"[-0.03995487093925476, -0.004647528287023306, ..."
4,output.txt,my usual  ve. I spent the morning at the docto...,24,"[0.005630828905850649, 0.00783389899879694, 0...."
...,...,...,...,...
302,output.txt,"us in trouble with the union, maybe even trigg...",15,"[-0.024327103048563004, -0.023813815787434578,..."
303,output.txt,creating some very bad press for us.”,8,"[-0.02393968403339386, 0.00013068846601527184,..."
304,output.txt,I nod automatically. “Restore critical busines...,13,"[-0.019007505849003792, -0.04307106137275696, ..."
305,output.txt,of the front-page news. Got it. anks.”,15,"[-0.02909829467535019, 0.0011510824551805854, ..."


In [78]:
# search embedded docs based on cosine similarity

# search through the reviews for a specific product
def search_docs(df, user_query, top_n=3, to_print=True):
    embedding = get_embedding(
        user_query,
        engine="embedding" # engine should be set to the deployment name you chose when you deployed the text-embedding-ada-002 (Version 2) model
    )
    df["similarities"] = df.ada_v2_embedding.apply(lambda x: cosine_similarity(x, embedding))

    res = (
        df.sort_values("similarities", ascending=False)
        .head(top_n)
    )
    if to_print:
        display(res)
    return res

df_similarities = df_embeddings.copy()

res = search_docs(df_similarities, "What is the role of Luke", top_n=4)

,FILE NAME,CONTENT,n_tokens,ada_v2_embedding,similarities
88,output.txt,"Luke was our , or Chief Information Oﬃcer. ...",26,"[0.011217995546758175, -0.007937762886285782, ...",0.831240
81,output.txt,Luke and Damon are no longer with the company....,20,"[0.010870774276554585, -0.007836912758648396, ...",0.772282
181,output.txt,made. You know Luke and Damon are no longer wi...,49,"[-0.007734533399343491, -0.019178904592990875,...",0.770837
0,output.txt,CHAPTER 1,3,"[0.015155577100813389, -0.01603473164141178, -...",0.749873


In [79]:
res.CONTENT.values[0]

'Luke was our \uf763\uf769\uf76f, or Chief Information Oﬃcer. Damon worked for him'

In [80]:
df_similarities

,FILE NAME,CONTENT,n_tokens,ada_v2_embedding,similarities
0,output.txt,CHAPTER 1,3,"[0.015155577100813389, -0.01603473164141178, -...",0.749873
1,output.txt,"• Tuesday, September 2",6,"[-0.005716212093830109, -0.010315271094441414,...",0.708647
2,output.txt,"“Bill Palmer here,” I say, answering my cell p...",19,"[-0.03608514368534088, -0.009047702886164188, ...",0.705206
3,output.txt,"I’m late, so I’m driving ten miles per hour ov...",19,"[-0.03995487093925476, -0.004647528287023306, ...",0.637889
4,output.txt,my usual  ve. I spent the morning at the docto...,24,"[0.005630828905850649, 0.00783389899879694, 0....",0.665755
...,...,...,...,...,...
302,output.txt,"us in trouble with the union, maybe even trigg...",15,"[-0.024327103048563004, -0.023813815787434578,...",0.705312
303,output.txt,creating some very bad press for us.”,8,"[-0.02393968403339386, 0.00013068846601527184,...",0.691157
304,output.txt,I nod automatically. “Restore critical busines...,13,"[-0.019007505849003792, -0.04307106137275696, ...",0.690030
305,output.txt,of the front-page news. Got it. anks.”,15,"[-0.02909829467535019, 0.0011510824551805854, ...",0.706686


In [82]:
def search_docs(df, user_query, top_n=3, to_print=True):
    embedding = get_embedding(
        user_query,
        model="embedding"
    )
    
    df_question = df_similarities.copy()
    df_question["similarities"] = df_similarities.ada_v2_embedding.apply(lambda x: cosine_similarity(x, embedding))

    res = (
        df_question.sort_values("similarities", ascending=False)
        .head(top_n)
    )
    return res

question = "What is the role of Luke"
res = search_docs(df, question, top_n=1)

context= res.CONTENT.values
completion_model='davinci-003'

initial_prompt = "The following is a conversation with an AI assistant. The assistant is helpful, creative, clever, and very friendly."

combined_prompt = initial_prompt + str(context) + "Q: " + question
response = openai.Completion.create(model=completion_model, prompt=combined_prompt, max_tokens=100)
ai_response = response['choices'][0]['text'].replace('\n', '').replace(' .', '.').strip()

print("\n"+ ai_response)

RetryError: RetryError[<Future at 0x7f9bffd32cd0 state=finished raised InvalidRequestError>]